#  Coursera Capstone Project

## *IBM Applied Data Science Capstone*

###  Opening a new restaurant in Ciudad Autonoma of Buenos Aires, Argentina

***

By: Sebastian Casali

<img src=http://www.telam.com.ar/advf/imagenes/2017/05/590c4b90da960_645x362.jpg>


# Introduction

For many people, food occupies an important place in the daily routine, more than anything on weekends. What better place to have a restaurant than in a world gastronomic center like the Autonomous City of Buenos Aires. As the Gastronomic Capital of Ibero-America 2017, Buenos Aires has a proposal of inexhaustible cuisine and gastronomic poles to taste the best recipes. A warm stew, a delicious barbecue or a delicious hot chocolate with churros are options to regain energy after a day full of activities on this winter vacation. It is not only the capital of Argentina, but it is one of the largest cities in South America due to the concentration of inhabitants it has.

***

# Problem

The objective of this Capstone Project is to analyze the different areas within the Autonomous City of Buenos Aires, in order to see where there is more concentration of clients and competition in the market. It is also intended to analyze the different gastronomic items in the areas, in order to be able to select a business that has a comparative advantage over the rest. In other words, the problem will be selecting a good location and a strategic restaurant item.

***

## Target Audience From This Project

This project is suitable for investors and entrepreneurs in the gastronomic field. Being such a large and recognized city in the world, it is a strategic point for restaurants due to the large number of tourists who visit it, and the high level of gastronomy in Argentines is an attraction known to all, mixing both Italian and Spanish culinary traits. 

***

## Data

#### To solve this problem, we will need the following data:

1. List of neighborhoods in Buenos Aires. This defines the scope of this project which is confined to the Ciudad Autonoma de Buenos Aires.
2. Latitude and Longitude cordinates of those neighborhoods. This is required in order to plot the map and also get the venue data.
3. Venue data, particularly data related to restaurants. We will use this data to perform clustering on neighborhoods.

#### Sources of data and methods to extract them:

We will use Wikipedia for extracting a list of neighborhoods from Buenos Aires, which has more than 50 of them. We will use Web Scraping techniques with the help ph Python request and beautifulsoup package. Then we will get the geographical coordinates of the neighborhoods using Python Geocoder package which will give us the latitude and longitude coordinates from the neighborhoods.
After that, we will use Foursquare API to get the venue data from those neighborhoods. Foursquare has one of the largest database of 105+ million places and is used by over 150,000 developers. Foursquare API will provide many categories of the venue data, we are particularly interested in the restaurants information. In this project, we will use many data science skills likeweb scraping, working with API, data cleaning, data wrangling, clustering and map visualization.

***

## Analzisis

#### Importing Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

### Importing Data From wikipedia and Work on it

In [5]:
url = "https://en.wikipedia.org/wiki/Neighbourhoods_of_Buenos_Aires"
pd.read_html(url, attrs={"class": "wikitable"})

[                   Name  Area in km²  Population  Commune
 0             Agronomía          2.1       13963       15
 1               Almagro          4.1      128206        5
 2             Balvanera          4.4      137521        3
 3              Barracas          7.6       73377        4
 4              Belgrano          6.8      126816       13
 5                 Boedo          2.6       45563        5
 6             Caballito          6.8      170309        6
 7             Chacarita          3.1       25778       15
 8               Coghlan          1.3       18021       12
 9            Colegiales          2.3       52391       13
 10         Constitución          2.1       41894        1
 11               Flores          7.8      142695        7
 12             Floresta          2.3       37247       10
 13              La Boca          3.1       43413        4
 14          La Paternal          2.2       19058       15
 15              Liniers          4.3       42083       

In [6]:
df = pd.read_html('https://en.wikipedia.org/wiki/Neighbourhoods_of_Buenos_Aires')[0]
df.head()

,Name,Area in km²,Population,Commune
0,Agronomía,2.1,13963,15
1,Almagro,4.1,128206,5
2,Balvanera,4.4,137521,3
3,Barracas,7.6,73377,4
4,Belgrano,6.8,126816,13


In [7]:
df.shape[0]

48

### Working with Foursquare

In [8]:
CLIENT_ID = "XREXEZYTVIN3WECWJIF3HRMEZNIDG1V2LIPMOSCWFRZ0VMH2" # your Foursquare ID
CLIENT_SECRET = "31EF1T1GJ34Y4FPQN0Z3UYO5EMYESUWYNX23MWDQUB44QEGP" # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XREXEZYTVIN3WECWJIF3HRMEZNIDG1V2LIPMOSCWFRZ0VMH2
CLIENT_SECRET:31EF1T1GJ34Y4FPQN0Z3UYO5EMYESUWYNX23MWDQUB44QEGP


In [9]:
address = 'Aráoz 1750, CABA, Buenos Aires'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-34.59052720408163 -58.42287520612245


### Searching for Italian competitors

In [10]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [11]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e931d381a4b0a001ba8e1c5'},
 'response': {'venues': [{'id': '4f65429ce4b0dc7720d1674e',
    'name': 'Orgullo Pizza & Pasta',
    'location': {'address': 'Paraguay 4062',
     'crossStreet': 'Araoz',
     'lat': -34.588027785438015,
     'lng': -58.41983357916842,
     'labeledLatLngs': [{'label': 'display',
       'lat': -34.588027785438015,
       'lng': -58.41983357916842}],
     'distance': 393,
     'cc': 'AR',
     'city': 'Buenos Aires',
     'state': 'Buenos Aires C.F.',
     'country': 'Argentina',
     'formattedAddress': ['Paraguay 4062 (Araoz)',
      'Buenos Aires',
      'Buenos Aires C.F.',
      'Argentina']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1586699453',
    'hasPerk': Fals

In [12]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4f65429ce4b0dc7720d1674e,Paraguay 4062,AR,Buenos Aires,Argentina,Araoz,393,"[Paraguay 4062 (Araoz), Buenos Aires, Buenos A...","[{'label': 'display', 'lat': -34.5880277854380...",-34.588028,-58.419834,Buenos Aires C.F.,Orgullo Pizza & Pasta,v-1586699453


In [13]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,Orgullo Pizza & Pasta,Pizza Place,Paraguay 4062,AR,Buenos Aires,Argentina,Araoz,393,"[Paraguay 4062 (Araoz), Buenos Aires, Buenos A...","[{'label': 'display', 'lat': -34.5880277854380...",-34.588028,-58.419834,Buenos Aires C.F.,4f65429ce4b0dc7720d1674e


### Maping the Pirncipal Competitor

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Searching for more competitors

In [15]:
latitude = -34.59052720408163
longitude = -58.42287520612245

In [16]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XREXEZYTVIN3WECWJIF3HRMEZNIDG1V2LIPMOSCWFRZ0VMH2&client_secret=31EF1T1GJ34Y4FPQN0Z3UYO5EMYESUWYNX23MWDQUB44QEGP&ll=-34.59052720408163,-58.42287520612245&v=20180604&radius=500&limit=30'

In [17]:
import requests

In [18]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

In [19]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5317e42a11d2d8dd97878d2d',
  'name': 'Teatro El Método Kairós',
  'location': {'address': 'El Salvador 4530',
   'crossStreet': 'esq. Av. Raúl Scalabrini Ortiz',
   'lat': -34.59059832364054,
   'lng': -58.42534529336995,
   'labeledLatLngs': [{'label': 'display',
     'lat': -34.59059832364054,
     'lng': -58.42534529336995}],
   'distance': 226,
   'postalCode': 'C1414BPH',
   'cc': 'AR',
   'city': 'Buenos Aires',
   'state': 'Buenos Aires C.F.',
   'country': 'Argentina',
   'formattedAddress': ['El Salvador 4530 (esq. Av. Raúl Scalabrini Ortiz)',
    'C1414BPH Palermo',
    'Buenos Aires C.F.',
    'Argentina']},
  'categories': [{'id': '4bf58dd8d48988d137941735',
    'name': 'Theater',
    'pluralName': 'Theaters',
    'shortName': 'Theater',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainm

In [20]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Teatro El Método Kairós,Theater,El Salvador 4530,AR,Buenos Aires,Argentina,esq. Av. Raúl Scalabrini Ortiz,226,[El Salvador 4530 (esq. Av. Raúl Scalabrini Or...,"[{'label': 'display', 'lat': -34.5905983236405...",-34.590598,-58.425345,NaN,C1414BPH,Buenos Aires C.F.,5317e42a11d2d8dd97878d2d
1,Cuervo Café,Coffee Shop,El Salvador 4580,AR,Buenos Aires,Argentina,e/ Av. Raúl Scalabrini Ortíz y Malabia,274,[El Salvador 4580 (e/ Av. Raúl Scalabrini Ortí...,"[{'label': 'display', 'lat': -34.5902348116501...",-34.590235,-58.425851,Palermo Viejo,C1414BPH,Buenos Aires C.F.,5957efd9029a55239dcf7e81
2,Rapa Nui,Chocolate Shop,Malabia 2014,AR,Buenos Aires,Argentina,esq. Soler,281,"[Malabia 2014 (esq. Soler), C1425EZB Buenos Ai...","[{'label': 'display', 'lat': -34.5880058928023...",-34.588006,-58.423064,Palermo Viejo,C1425EZB,Buenos Aires C.F.,5a4441f932b61d131d125b9d
3,Casa Coupage,Argentinian Restaurant,Francisco Acuña de Figueroa 1790,AR,Buenos Aires,Argentina,esq. Soler,228,[Francisco Acuña de Figueroa 1790 (esq. Soler)...,"[{'label': 'display', 'lat': -34.5902673952961...",-34.590267,-58.420401,Palermo Viejo,C1180ABH,Buenos Aires C.F.,4d2526dcb9796dcbb640a639
4,Nicky New York Sushi,Sushi Restaurant,Malabia 1764,AR,Buenos Aires,Argentina,e/ El Salvador y Costa Rica,256,"[Malabia 1764 (e/ El Salvador y Costa Rica), C...","[{'label': 'display', 'lat': -34.5896742734276...",-34.589674,-58.425479,NaN,C1414DMJ,Buenos Aires C.F.,518d9c89498e5bdd30372921
5,The Harrison Speakeasy,Speakeasy,Malabia 1764,AR,Buenos Aires,Argentina,esq. Costa Rica,279,"[Malabia 1764 (esq. Costa Rica), C1414DMJ Buen...","[{'label': 'display', 'lat': -34.5895491508849...",-34.589549,-58.425688,NaN,C1414DMJ,Buenos Aires C.F.,51df6223498ea627237bcd8d
6,Proper,Restaurant,Aráoz 1676,AR,Buenos Aires,Argentina,esq. El Salvador,74,"[Aráoz 1676 (esq. El Salvador), C1414DQH Paler...","[{'label': 'display', 'lat': -34.5909777494236...",-34.590978,-58.423472,Palermo Viejo,C1414DQH,Buenos Aires C.F.,56e37399cd10fabb7a9bf0eb
7,VinoTinto Cocina Andante,Restaurant,Julián Álvarez 1602,AR,Buenos Aires,Argentina,esq. El Salvador,196,"[Julián Álvarez 1602 (esq. El Salvador), C1414...","[{'label': 'display', 'lat': -34.5922430805430...",-34.592243,-58.423364,Palermo Viejo,C1414DSH,Buenos Aires C.F.,553144c2498e3e975d783049
8,Gran Dabbang Cafe,Indian Restaurant,Av. Raúl Scalabrini Ortiz 1543,AR,Buenos Aires,Argentina,e/ Honduras y El Salvador,260,[Av. Raúl Scalabrini Ortiz 1543 (e/ Honduras y...,"[{'label': 'display', 'lat': -34.5913167536271...",-34.591317,-58.425548,Palermo Viejo,C1414DOC,Buenos Aires C.F.,53b2096c498e59958f34de99
9,Cosi Mi Piace,Pizza Place,El Salvador 4618,AR,Buenos Aires,Argentina,e/ Malabia y Armenia,331,"[El Salvador 4618 (e/ Malabia y Armenia), C141...","[{'label': 'display', 'lat': -34.5900074300563...",-34.590007,-58.426440,Palermo Viejo,C1414BPJ,Buenos Aires C.F.,56fb201a498eb4dfe2922f62


In [21]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Teatro El Método Kairós,Theater,El Salvador 4530,AR,Buenos Aires,Argentina,esq. Av. Raúl Scalabrini Ortiz,226,[El Salvador 4530 (esq. Av. Raúl Scalabrini Or...,"[{'label': 'display', 'lat': -34.5905983236405...",-34.590598,-58.425345,NaN,C1414BPH,Buenos Aires C.F.,5317e42a11d2d8dd97878d2d
1,Cuervo Café,Coffee Shop,El Salvador 4580,AR,Buenos Aires,Argentina,e/ Av. Raúl Scalabrini Ortíz y Malabia,274,[El Salvador 4580 (e/ Av. Raúl Scalabrini Ortí...,"[{'label': 'display', 'lat': -34.5902348116501...",-34.590235,-58.425851,Palermo Viejo,C1414BPH,Buenos Aires C.F.,5957efd9029a55239dcf7e81
2,Rapa Nui,Chocolate Shop,Malabia 2014,AR,Buenos Aires,Argentina,esq. Soler,281,"[Malabia 2014 (esq. Soler), C1425EZB Buenos Ai...","[{'label': 'display', 'lat': -34.5880058928023...",-34.588006,-58.423064,Palermo Viejo,C1425EZB,Buenos Aires C.F.,5a4441f932b61d131d125b9d
3,Casa Coupage,Argentinian Restaurant,Francisco Acuña de Figueroa 1790,AR,Buenos Aires,Argentina,esq. Soler,228,[Francisco Acuña de Figueroa 1790 (esq. Soler)...,"[{'label': 'display', 'lat': -34.5902673952961...",-34.590267,-58.420401,Palermo Viejo,C1180ABH,Buenos Aires C.F.,4d2526dcb9796dcbb640a639
4,Nicky New York Sushi,Sushi Restaurant,Malabia 1764,AR,Buenos Aires,Argentina,e/ El Salvador y Costa Rica,256,"[Malabia 1764 (e/ El Salvador y Costa Rica), C...","[{'label': 'display', 'lat': -34.5896742734276...",-34.589674,-58.425479,NaN,C1414DMJ,Buenos Aires C.F.,518d9c89498e5bdd30372921
5,The Harrison Speakeasy,Speakeasy,Malabia 1764,AR,Buenos Aires,Argentina,esq. Costa Rica,279,"[Malabia 1764 (esq. Costa Rica), C1414DMJ Buen...","[{'label': 'display', 'lat': -34.5895491508849...",-34.589549,-58.425688,NaN,C1414DMJ,Buenos Aires C.F.,51df6223498ea627237bcd8d
6,Proper,Restaurant,Aráoz 1676,AR,Buenos Aires,Argentina,esq. El Salvador,74,"[Aráoz 1676 (esq. El Salvador), C1414DQH Paler...","[{'label': 'display', 'lat': -34.5909777494236...",-34.590978,-58.423472,Palermo Viejo,C1414DQH,Buenos Aires C.F.,56e37399cd10fabb7a9bf0eb
7,VinoTinto Cocina Andante,Restaurant,Julián Álvarez 1602,AR,Buenos Aires,Argentina,esq. El Salvador,196,"[Julián Álvarez 1602 (esq. El Salvador), C1414...","[{'label': 'display', 'lat': -34.5922430805430...",-34.592243,-58.423364,Palermo Viejo,C1414DSH,Buenos Aires C.F.,553144c2498e3e975d783049
8,Gran Dabbang Cafe,Indian Restaurant,Av. Raúl Scalabrini Ortiz 1543,AR,Buenos Aires,Argentina,e/ Honduras y El Salvador,260,[Av. Raúl Scalabrini Ortiz 1543 (e/ Honduras y...,"[{'label': 'display', 'lat': -34.5913167536271...",-34.591317,-58.425548,Palermo Viejo,C1414DOC,Buenos Aires C.F.,53b2096c498e59958f34de99
9,Cosi Mi Piace,Pizza Place,El Salvador 4618,AR,Buenos Aires,Argentina,e/ Malabia y Armenia,331,"[El Salvador 4618 (e/ Malabia y Armenia), C141...","[{'label': 'display', 'lat': -34.5900074300563...",-34.590007,-58.426440,Palermo Viejo,C1414BPJ,Buenos Aires C.F.,56fb201a498eb4dfe2922f62


### Searching for Venues

In [22]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e931c75d03993001bff4b24'},
 'response': {'venues': []}}

In [23]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e931b6f40a7ea001b27a1ea'},
 'response': {'venues': []}}

In [24]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [25]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [26]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [27]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

### Analizing competitors

In [28]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Teatro El Método Kairós,Theater,El Salvador 4530,AR,Buenos Aires,Argentina,esq. Av. Raúl Scalabrini Ortiz,226,[El Salvador 4530 (esq. Av. Raúl Scalabrini Or...,"[{'label': 'display', 'lat': -34.5905983236405...",-34.590598,-58.425345,NaN,C1414BPH,Buenos Aires C.F.,5317e42a11d2d8dd97878d2d
1,Cuervo Café,Coffee Shop,El Salvador 4580,AR,Buenos Aires,Argentina,e/ Av. Raúl Scalabrini Ortíz y Malabia,274,[El Salvador 4580 (e/ Av. Raúl Scalabrini Ortí...,"[{'label': 'display', 'lat': -34.5902348116501...",-34.590235,-58.425851,Palermo Viejo,C1414BPH,Buenos Aires C.F.,5957efd9029a55239dcf7e81
2,Rapa Nui,Chocolate Shop,Malabia 2014,AR,Buenos Aires,Argentina,esq. Soler,281,"[Malabia 2014 (esq. Soler), C1425EZB Buenos Ai...","[{'label': 'display', 'lat': -34.5880058928023...",-34.588006,-58.423064,Palermo Viejo,C1425EZB,Buenos Aires C.F.,5a4441f932b61d131d125b9d
3,Casa Coupage,Argentinian Restaurant,Francisco Acuña de Figueroa 1790,AR,Buenos Aires,Argentina,esq. Soler,228,[Francisco Acuña de Figueroa 1790 (esq. Soler)...,"[{'label': 'display', 'lat': -34.5902673952961...",-34.590267,-58.420401,Palermo Viejo,C1180ABH,Buenos Aires C.F.,4d2526dcb9796dcbb640a639
4,Nicky New York Sushi,Sushi Restaurant,Malabia 1764,AR,Buenos Aires,Argentina,e/ El Salvador y Costa Rica,256,"[Malabia 1764 (e/ El Salvador y Costa Rica), C...","[{'label': 'display', 'lat': -34.5896742734276...",-34.589674,-58.425479,NaN,C1414DMJ,Buenos Aires C.F.,518d9c89498e5bdd30372921
5,The Harrison Speakeasy,Speakeasy,Malabia 1764,AR,Buenos Aires,Argentina,esq. Costa Rica,279,"[Malabia 1764 (esq. Costa Rica), C1414DMJ Buen...","[{'label': 'display', 'lat': -34.5895491508849...",-34.589549,-58.425688,NaN,C1414DMJ,Buenos Aires C.F.,51df6223498ea627237bcd8d
6,Proper,Restaurant,Aráoz 1676,AR,Buenos Aires,Argentina,esq. El Salvador,74,"[Aráoz 1676 (esq. El Salvador), C1414DQH Paler...","[{'label': 'display', 'lat': -34.5909777494236...",-34.590978,-58.423472,Palermo Viejo,C1414DQH,Buenos Aires C.F.,56e37399cd10fabb7a9bf0eb
7,VinoTinto Cocina Andante,Restaurant,Julián Álvarez 1602,AR,Buenos Aires,Argentina,esq. El Salvador,196,"[Julián Álvarez 1602 (esq. El Salvador), C1414...","[{'label': 'display', 'lat': -34.5922430805430...",-34.592243,-58.423364,Palermo Viejo,C1414DSH,Buenos Aires C.F.,553144c2498e3e975d783049
8,Gran Dabbang Cafe,Indian Restaurant,Av. Raúl Scalabrini Ortiz 1543,AR,Buenos Aires,Argentina,e/ Honduras y El Salvador,260,[Av. Raúl Scalabrini Ortiz 1543 (e/ Honduras y...,"[{'label': 'display', 'lat': -34.5913167536271...",-34.591317,-58.425548,Palermo Viejo,C1414DOC,Buenos Aires C.F.,53b2096c498e59958f34de99
9,Cosi Mi Piace,Pizza Place,El Salvador 4618,AR,Buenos Aires,Argentina,e/ Malabia y Armenia,331,"[El Salvador 4618 (e/ Malabia y Armenia), C141...","[{'label': 'display', 'lat': -34.5900074300563...",-34.590007,-58.426440,Palermo Viejo,C1414BPJ,Buenos Aires C.F.,56fb201a498eb4dfe2922f62


In [29]:
df = dataframe_filtered
df.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Teatro El Método Kairós,Theater,El Salvador 4530,AR,Buenos Aires,Argentina,esq. Av. Raúl Scalabrini Ortiz,226,[El Salvador 4530 (esq. Av. Raúl Scalabrini Or...,"[{'label': 'display', 'lat': -34.5905983236405...",-34.590598,-58.425345,NaN,C1414BPH,Buenos Aires C.F.,5317e42a11d2d8dd97878d2d
1,Cuervo Café,Coffee Shop,El Salvador 4580,AR,Buenos Aires,Argentina,e/ Av. Raúl Scalabrini Ortíz y Malabia,274,[El Salvador 4580 (e/ Av. Raúl Scalabrini Ortí...,"[{'label': 'display', 'lat': -34.5902348116501...",-34.590235,-58.425851,Palermo Viejo,C1414BPH,Buenos Aires C.F.,5957efd9029a55239dcf7e81
2,Rapa Nui,Chocolate Shop,Malabia 2014,AR,Buenos Aires,Argentina,esq. Soler,281,"[Malabia 2014 (esq. Soler), C1425EZB Buenos Ai...","[{'label': 'display', 'lat': -34.5880058928023...",-34.588006,-58.423064,Palermo Viejo,C1425EZB,Buenos Aires C.F.,5a4441f932b61d131d125b9d
3,Casa Coupage,Argentinian Restaurant,Francisco Acuña de Figueroa 1790,AR,Buenos Aires,Argentina,esq. Soler,228,[Francisco Acuña de Figueroa 1790 (esq. Soler)...,"[{'label': 'display', 'lat': -34.5902673952961...",-34.590267,-58.420401,Palermo Viejo,C1180ABH,Buenos Aires C.F.,4d2526dcb9796dcbb640a639
4,Nicky New York Sushi,Sushi Restaurant,Malabia 1764,AR,Buenos Aires,Argentina,e/ El Salvador y Costa Rica,256,"[Malabia 1764 (e/ El Salvador y Costa Rica), C...","[{'label': 'display', 'lat': -34.5896742734276...",-34.589674,-58.425479,NaN,C1414DMJ,Buenos Aires C.F.,518d9c89498e5bdd30372921


In [30]:
df = df.drop(['formattedAddress'], axis=1)
df = df.drop(['labeledLatLngs'], axis=1)
df = df.drop(['crossStreet'], axis=1)
df = df.drop(['distance'], axis=1)
df = df.drop(['cc'], axis=1)
df = df.drop(['id'], axis=1)
df = df.drop(['categories'], axis=1)
df = df.drop(['address'], axis=1)
df = df.drop(['country'], axis=1)
df = df.drop(['state'], axis=1)
df = df.drop(['postalCode'], axis=1)

df.head()

,name,city,lat,lng,neighborhood
0,Teatro El Método Kairós,Buenos Aires,-34.590598,-58.425345,NaN
1,Cuervo Café,Buenos Aires,-34.590235,-58.425851,Palermo Viejo
2,Rapa Nui,Buenos Aires,-34.588006,-58.423064,Palermo Viejo
3,Casa Coupage,Buenos Aires,-34.590267,-58.420401,Palermo Viejo
4,Nicky New York Sushi,Buenos Aires,-34.589674,-58.425479,NaN


### Seing Our Potencial Location

In [31]:
# create map of New York using latitude and longitude values
map_caba = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, df in zip(df['lat'], df['lng'], df['city'], df['neighborhood']):
    label = '{}, {}'.format(df, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_caba)  
    
map_caba

***

# Conclusion

##The Potencial Location of the restaurant in Ciudad Autonoma de Buenos Aires seems to be a good one. There is no much competitors in the area, and there is just one Italian restaurant place near our location. So, in conclusion, our restaurant could be one of Italian food, such as pasta or pizza because of the potencial customers we could have in this area.